# Classifying movie reviews
**A binary classification example**

Two-class classification, or binary classification, may be the most widely applied kind of machine-learning problem.
In this example, movie reviews will be classified as positive or negative, based on the text content of the reviews.

## The IMDB dataset
You’ll work with the [IMDB dataset](https://www.imdb.com/): a set of 50,000 highly polarized reviews from the Internet Movie Database. They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews.

In [ ]:
from keras.datasets import imdb
(train_data, train_labels),(test_data,test_labels) = imdb.load_data(num_words=10000)

* `num_words=10000` means you'll only keep the top 10,000 most frequently ocurring words in the training data. Rare words will be discarded.

* `train_data` an `test_data` are lists of reviews. Each review is a list of word indices ranked by frequency.

* `train_labels` and `test_labels` are lists of 0s and 1s,where 0 stands for *negative* and 1 for *positive*.

## Preparing the data
You have to turn the lists into tensors. There are 2 ways to do that:
* Pad the lists so that they all have the same length, turn them into an integer tensor of shape `(samples, word_indices)`, and use as the first layer in the network a layer capable of handling such integer tensors. (The *Embedding* layer).
* One-hot encode the lists and turn them into vectors of 0s and 1s. This would mean, for instance, turning the sequence `[3,5]` into a 10,000-dimentional vector that would be all 0s except for `[3,5]`. The use a *Dense* layer as first layer, capable of handling floating-point vector data.

In [ ]:
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence]=1.
    return results
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [ ]:
x_train[0]

In [ ]:
x_test[0]